
# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [4]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import numpy as np
#psycopg2 has problems with understanding numpy data types
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)



In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [7]:
song_files = get_files("data/song_data")
song_files

['/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json',
 '/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/home/workspace/data/song_data/A/A/B/TRAABNV128F425CEE1.json',
 '/home/workspace/data/song_data/A/A/B/TRAABRB128F9306DD5.json',
 '/home/workspace/data/song_data/A/A/B/TRAABLR128F423B7E3.json',
 '/home/workspace/data/song_data/A/A/B/TRAABDL12903CAABBA.json',
 '/home/workspace/data/so

In [8]:
filepath = song_files[0]

In [9]:
df = pd.read_json (filepath,lines=True)
df.head()
#df.dtypes

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
song_data = df[['song_id','title','artist_id','year','duration']].iloc[0].values.tolist()
#song_data = song_data.iloc[0].values.tolist()
song_data


['SOMZWCG12A8C13C480',
 "I Didn't Mean To",
 'ARD7TVE1187B99BFB1',
 0,
 218.93179000000001]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
cur.execute(song_table_insert, song_data)
conn.commit()


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [15]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].iloc[0].values.tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [16]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [17]:
log_files = get_files("data/log_data")
log_files

['/home/workspace/data/log_data/2018/11/2018-11-30-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-23-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-24-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-25-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-15-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-26-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-04-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.json',
 '/home/workspace/data/lo

In [18]:
filepath = log_files[0]

In [19]:
df = pd.read_json (filepath,lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [20]:
df = df[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [22]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [23]:
time_data = [t,t.dt.hour,t.dt.day,t.dt.week,t.dt.month,t.dt.year,t.dt.weekday]
column_labels = ['timestamp','hour','day','week','month','year','weekday']
time_dict = {column_labels[i]: time_data[i] for i in range(len(column_labels))} 
time_dict

{'timestamp': 0     2018-11-30 00:22:07.796
 1     2018-11-30 01:08:41.796
 2     2018-11-30 01:12:48.796
 3     2018-11-30 01:17:05.796
 4     2018-11-30 01:20:56.796
 5     2018-11-30 01:24:32.796
 6     2018-11-30 01:28:19.796
 7     2018-11-30 01:32:00.796
 9     2018-11-30 01:38:17.796
 10    2018-11-30 01:39:53.796
 11    2018-11-30 01:44:10.796
 12    2018-11-30 01:48:36.796
 13    2018-11-30 01:50:28.796
 14    2018-11-30 01:55:35.796
 15    2018-11-30 02:01:08.796
 16    2018-11-30 02:05:24.796
 17    2018-11-30 02:10:21.796
 18    2018-11-30 02:15:54.796
 19    2018-11-30 02:19:19.796
 23    2018-11-30 03:06:30.796
 24    2018-11-30 03:10:30.796
 25    2018-11-30 03:13:03.796
 26    2018-11-30 03:15:41.796
 27    2018-11-30 03:21:09.796
 30    2018-11-30 03:29:23.796
 31    2018-11-30 03:32:46.796
 33    2018-11-30 03:37:24.796
 35    2018-11-30 03:39:31.796
 36    2018-11-30 03:44:28.796
 38    2018-11-30 03:49:50.796
                 ...          
 356   2018-11-30 17:49:45

In [24]:
time_df =  pd.DataFrame.from_dict(time_dict, orient='columns')
time_df.head(11)

,timestamp,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4
3,2018-11-30 01:17:05.796,1,30,48,11,2018,4
4,2018-11-30 01:20:56.796,1,30,48,11,2018,4
5,2018-11-30 01:24:32.796,1,30,48,11,2018,4
6,2018-11-30 01:28:19.796,1,30,48,11,2018,4
7,2018-11-30 01:32:00.796,1,30,48,11,2018,4
9,2018-11-30 01:38:17.796,1,30,48,11,2018,4
10,2018-11-30 01:39:53.796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [25]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [26]:
user_df = df[['userId','firstName','lastName','gender','level']].drop_duplicates(subset = ['userId'])

user_df.head()

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
23,86,Aiden,Hess,M,free
30,24,Layla,Griffin,F,paid
40,26,Ryan,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [27]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [28]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (index,pd.to_datetime(row.ts, unit='ms'),row.userId,row.level,songid, artistid,row.sessionId,row.location,row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [29]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.